# Proximal Policy Optimization (PPO) Tutorial

## 1. Environment Preparation




### 1.1 Download Packages for BipedalWalker-v3

In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install colabgymrender==1.0.2
!pip install box2d-py
!pip install 'gym[Box2D]'

### 1.2 Mount Drive and Set Project Path

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
project_root = '/content/drive/My Drive/ppo_tutorial/'
sys.path.append(project_root)

Mounted at /content/drive


### 1.3 Test the BipedalWalker-v3 Environment 

In [3]:
import os
import gym
import torch
import torch.nn as nn
import numpy as np
from colabgymrender.recorder import Recorder

env = gym.make('BipedalWalker-v3')
s_dim = env.observation_space.shape[0]
a_dim = env.action_space.shape[0]
print(s_dim)
print(a_dim)

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1810432/45929032 bytes (3.9%)4513792/45929032 bytes (9.8%)7520256/45929032 bytes (16.4%)10125312/45929032 bytes (22.0%)12976128/45929032 bytes (28.3%)16023552/45929032 bytes (34.9%)18882560/45929032 bytes (41.1%)21757952/45929032 bytes (47.4%)24657920/45929032 bytes (53.7%)27631616/45929032 bytes (60.2%)30466048/45929032 bytes (66.3%)33300480/45929032 bytes (72.5%)3

## 2. Policy Network & Value Network Construction

### 2.1 Diagonal Gaussian Distribution Module 

In [4]:
#AddBias module
class AddBias(nn.Module):
    def __init__(self, bias):
        super(AddBias, self).__init__()
        self._bias = nn.Parameter(bias.unsqueeze(1))
    
    def forward(self, x):
        bias = self._bias.t().view(1, -1)
        return x + bias

#Gaussian distribution with given mean & std.
class FixedNormal(torch.distributions.Normal):
    def log_probs(self, x):
        return super().log_prob(x).sum(-1)
    
    def entropy(self):
        return super().entropy().sum(-1)

    def mode(self):
        return self.mean

#Diagonal Gaussian module
class DiagGaussian(nn.Module):
    def __init__(self, inp_dim, out_dim):
        super(DiagGaussian, self).__init__()
        self.fc_mean = nn.Linear(inp_dim, out_dim)
        self.b_logstd = AddBias(torch.zeros(out_dim))
    
    def forward(self, x):
        mean = self.fc_mean(x)
        logstd = self.b_logstd(torch.zeros_like(mean))
        return FixedNormal(mean, logstd.exp())

### 2.2 Policy Network & Value Network Module

In [5]:
#Policy Network
class PolicyNet(nn.Module):
    #Constructor
    def __init__(self, s_dim, a_dim):
        super(PolicyNet, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(s_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU()
        )
        self.dist = DiagGaussian(128, a_dim)
    
    #Forward pass
    def forward(self, state, deterministic=False):
        feature = self.main(state)
        dist = self.dist(feature)

        if deterministic:
            action = dist.mode()
        else:
            action = dist.sample()
        
        return action, dist.log_probs(action)
    
    #Choose an action (stochastically or deterministically)
    def choose_action(self, state, deterministic=False):
        feature = self.main(state)
        dist = self.dist(feature)

        if deterministic:
            return dist.mode()

        return dist.sample()
    
    #Evaluate a state-action pair (output log-prob. & entropy)
    def evaluate(self, state, action):
        feature = self.main(state)
        dist = self.dist(feature)
        return dist.log_probs(action), dist.entropy()

#Value Network
class ValueNet(nn.Module):
    #Constructor
    def __init__(self, s_dim):
        super(ValueNet, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(s_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
    
    #Forward pass
    def forward(self, state):
        return self.main(state)[:, 0]

### 2.3 Create Policy Network & Value Network

In [6]:
policy_net = PolicyNet(s_dim, a_dim)
value_net = ValueNet(s_dim)
print(policy_net)
print(value_net)

PolicyNet(
  (main): Sequential(
    (0): Linear(in_features=24, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
  )
  (dist): DiagGaussian(
    (fc_mean): Linear(in_features=128, out_features=4, bias=True)
    (b_logstd): AddBias()
  )
)
ValueNet(
  (main): Sequential(
    (0): Linear(in_features=24, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
  )
)


## 3. Environment Runner Construction

### 3.1 EnvRunner Class

In [7]:
class EnvRunner:
    #Constructor
    def __init__(self, s_dim, a_dim, gamma=0.99, lamb=0.95, max_step=2048, device='cpu'):
        self.s_dim = s_dim
        self.a_dim = a_dim
        self.gamma = gamma
        self.lamb = lamb
        self.max_step = max_step
        self.device = device

        #Storages (state, action, value, reward, a_logp)
        self.mb_states = np.zeros((self.max_step, self.s_dim), dtype=np.float32)
        self.mb_actions = np.zeros((self.max_step, self.a_dim), dtype=np.float32)
        self.mb_values = np.zeros((self.max_step,), dtype=np.float32)
        self.mb_rewards = np.zeros((self.max_step,), dtype=np.float32)
        self.mb_a_logps = np.zeros((self.max_step,), dtype=np.float32)
    
    #Compute discounted return
    def compute_discounted_return(self, rewards, last_value, gamma=0.99):
        returns = np.zeros_like(rewards)
        n_step = len(rewards)

        for t in reversed(range(n_step)):
            if t == n_step - 1:
                returns[t] = rewards[t] + gamma * last_value
            else:
                returns[t] = rewards[t] + gamma * returns[t+1]

        return returns
    
    #Compute generalized advantage estimation
    def compute_gae(self, rewards, values, last_value, gamma=0.99, lamb=0.95):
        advs = np.zeros_like(rewards)
        n_step = len(rewards)
        last_gae_lam = 0.0

        for t in reversed(range(n_step)):
            if t == n_step - 1:
                next_value = last_value
            else:
                next_value = values[t+1]

            delta = rewards[t] + gamma*next_value - values[t]
            advs[t] = last_gae_lam = delta + gamma*lamb*last_gae_lam

        return advs + values

    #Run an episode using the policy net & value net
    def run(self, env, policy_net, value_net):
        #Run an episode
        state = env.reset()   #Initial state
        episode_len = self.max_step

        for step in range(self.max_step):
            state_tensor = torch.tensor(np.expand_dims(state, axis=0), dtype=torch.float32, device=self.device)
            action, a_logp = policy_net(state_tensor)
            value = value_net(state_tensor)

            action = action.cpu().numpy()[0]
            a_logp = a_logp.cpu().numpy()
            value  = value.cpu().numpy()

            self.mb_states[step] = state
            self.mb_actions[step] = action
            self.mb_a_logps[step] = a_logp
            self.mb_values[step] = value

            state, reward, done, info = env.step(action)
            self.mb_rewards[step] = reward

            if done:
                episode_len = step + 1
                break
        
        #Compute returns
        last_value = value_net(
            torch.tensor(np.expand_dims(state, axis=0), dtype=torch.float32, device=self.device)
        ).cpu().numpy()

        mb_returns = self.compute_discounted_return(
            self.mb_rewards[:episode_len],
            last_value,
            self.lamb
        )
        '''
        mb_returns = self.compute_gae(
            self.mb_rewards[:episode_len],
            self.mb_values[:episode_len],
            last_value,
            self.gamma,
            self.lamb
        )
        '''

        return self.mb_states[:episode_len], \
                self.mb_actions[:episode_len], \
                self.mb_a_logps[:episode_len], \
                self.mb_values[:episode_len], \
                mb_returns, \
                self.mb_rewards[:episode_len]

### 3.2 Create EnvRunner

In [8]:
runner = EnvRunner(s_dim, a_dim)

## 4. PPO Algorithm

### 4.1 PPO Class

In [9]:
class PPO:
    #Constructor
    def __init__(self, policy_net, value_net, lr=1e-4, max_grad_norm=0.5, ent_weight=0.01, clip_val=0.2, sample_n_epoch=4, sample_mb_size=64, device='cpu'):
        self.policy_net = policy_net
        self.value_net = value_net
        self.max_grad_norm = max_grad_norm
        self.ent_weight = ent_weight
        self.clip_val = clip_val
        self.sample_n_epoch = sample_n_epoch
        self.sample_mb_size = sample_mb_size
        self.device = device
        self.opt_polcy = torch.optim.Adam(policy_net.parameters(), lr)
        self.opt_value = torch.optim.Adam(value_net.parameters(), lr)
    
    #Train the policy net & value net using PPO
    def train(self, mb_states, mb_actions, mb_old_values, mb_advs, mb_returns, mb_old_a_logps):
        #Convert numpy array to tensor
        mb_states = torch.from_numpy(mb_states).to(self.device)
        mb_actions = torch.from_numpy(mb_actions).to(self.device)
        mb_old_values = torch.from_numpy(mb_old_values).to(self.device)
        mb_advs = torch.from_numpy(mb_advs).to(self.device)
        mb_returns = torch.from_numpy(mb_returns).to(self.device)
        mb_old_a_logps = torch.from_numpy(mb_old_a_logps).to(self.device)
        episode_length = len(mb_states)
        rand_idx = np.arange(episode_length)
        sample_n_mb = episode_length // self.sample_mb_size

        if sample_n_mb <= 0:
            sample_mb_size = episode_length
            sample_n_mb = 1
        else:
            sample_mb_size = self.sample_mb_size

        for i in range(self.sample_n_epoch):
            np.random.shuffle(rand_idx)

            for j in range(sample_n_mb):
                #Randomly sample a batch for training
                sample_idx = rand_idx[j*sample_mb_size : (j+1)*sample_mb_size]
                sample_states = mb_states[sample_idx]
                sample_actions = mb_actions[sample_idx]
                sample_old_values = mb_old_values[sample_idx]
                sample_advs = mb_advs[sample_idx]
                sample_returns = mb_returns[sample_idx]
                sample_old_a_logps = mb_old_a_logps[sample_idx]

                sample_a_logps, sample_ents = self.policy_net.evaluate(sample_states, sample_actions)
                sample_values = self.value_net(sample_states)
                ent = sample_ents.mean()

                #Compute value loss
                v_pred_clip = sample_old_values + torch.clamp(sample_values - sample_old_values, -self.clip_val, self.clip_val)
                v_loss1 = (sample_returns - sample_values).pow(2)
                v_loss2 = (sample_returns - v_pred_clip).pow(2)
                v_loss = torch.max(v_loss1, v_loss2).mean()

                #Compute policy gradient loss
                ratio = (sample_a_logps - sample_old_a_logps).exp()
                pg_loss1 = -sample_advs * ratio
                pg_loss2 = -sample_advs * torch.clamp(ratio, 1.0-self.clip_val, 1.0+self.clip_val)
                pg_loss = torch.max(pg_loss1, pg_loss2).mean() - self.ent_weight*ent

                #Train actor
                self.opt_polcy.zero_grad()
                pg_loss.backward()
                nn.utils.clip_grad_norm_(self.policy_net.parameters(), self.max_grad_norm)
                self.opt_polcy.step()

                #Train critic
                self.opt_value.zero_grad()
                v_loss.backward()
                nn.utils.clip_grad_norm_(self.value_net.parameters(), self.max_grad_norm)
                self.opt_value.step()

        return pg_loss.item(), v_loss.item(), ent.item()

### 4.2 Create PPO Agent

In [10]:
agent = PPO(policy_net, value_net)

## 5. Training and Testing Process

### 5.1 Play an Episode for Evaluation

In [11]:
def play(policy_net):
    render_env = Recorder(gym.make('BipedalWalker-v3'), project_root + '/video')

    with torch.no_grad():
        state = render_env.reset()
        total_reward = 0
        length = 0

        while True:
            #render_env.render()
            state_tensor = torch.tensor(np.expand_dims(state, axis=0), dtype=torch.float32, device='cpu')
            action = policy_net.choose_action(state_tensor, deterministic=True).cpu().numpy()
            state, reward, done, info = render_env.step(action[0])
            total_reward += reward
            length += 1

            if done:
                print("[Evaluation] Total reward = {:.6f}, length = {:d}".format(total_reward, length), flush=True)
                break
    render_env.play()
    render_env.close()

In [12]:
play(policy_net)

[Evaluation] Total reward = -111.058200, length = 73


100%|██████████| 75/75 [00:00<00:00, 259.79it/s]


### 5.2 Train the Networks using PPO

In [13]:
def train(env, runner, policy_net, value_net, agent, max_episode=5000):
    mean_total_reward = 0
    mean_length = 0
    save_dir = project_root + '/save'

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    for i in range(max_episode):
        #Run an episode to collect data
        with torch.no_grad():
            mb_states, mb_actions, mb_old_a_logps, mb_values, mb_returns, mb_rewards = runner.run(env, policy_net, value_net)
            mb_advs = mb_returns - mb_values
            mb_advs = (mb_advs - mb_advs.mean()) / (mb_advs.std() + 1e-6)
        
        #Train the model using the collected data
        pg_loss, v_loss, ent = agent.train(mb_states, mb_actions, mb_values, mb_advs, mb_returns, mb_old_a_logps)
        mean_total_reward += mb_rewards.sum()
        mean_length += len(mb_states)
        print("[Episode {:4d}] total reward = {:.6f}, length = {:d}".format(i, mb_rewards.sum(), len(mb_states)))

        #Show the current result & save the model
        if i % 200 == 0:
            print("\n[{:5d} / {:5d}]".format(i, max_episode))
            print("----------------------------------")
            print("actor loss = {:.6f}".format(pg_loss))
            print("critic loss = {:.6f}".format(v_loss))
            print("entropy = {:.6f}".format(ent))
            print("mean return = {:.6f}".format(mean_total_reward / 100))
            print("mean length = {:.2f}".format(mean_length / 100))
            print("\nSaving the model ... ", end="")
            torch.save({
                "it": i,
                "PolicyNet": policy_net.state_dict(),
                "ValueNet": value_net.state_dict()
            }, os.path.join(save_dir, "model.pt"))
            print("Done.")
            print()
            play(policy_net)
            mean_total_reward = 0
            mean_length = 0

In [14]:
train(env, runner, policy_net, value_net, agent)
env.close()

[Episode    0] total reward = -101.405266, length = 1600

[    0 /  5000]
----------------------------------
actor loss = 0.056528
critic loss = 1.490289
entropy = 5.671003
mean return = -1.014053
mean length = 16.00

Saving the model ... Done.

[Evaluation] Total reward = -111.704622, length = 80


 99%|█████████▉| 81/82 [00:00<00:00, 262.01it/s]


[Episode    1] total reward = -114.313126, length = 1600
[Episode    2] total reward = -109.681152, length = 55
[Episode    3] total reward = -95.253601, length = 1600
[Episode    4] total reward = -120.235931, length = 102
[Episode    5] total reward = -115.665123, length = 106
[Episode    6] total reward = -110.038933, length = 100
[Episode    7] total reward = -113.464943, length = 174
[Episode    8] total reward = -110.747955, length = 57
[Episode    9] total reward = -114.234375, length = 76
[Episode   10] total reward = -109.079987, length = 48
[Episode   11] total reward = -108.891266, length = 43
[Episode   12] total reward = -103.560921, length = 78
[Episode   13] total reward = -111.903427, length = 71
[Episode   14] total reward = -101.470703, length = 1600
[Episode   15] total reward = -113.386108, length = 94
[Episode   16] total reward = -117.731384, length = 76
[Episode   17] total reward = -100.212234, length = 1600
[Episode   18] total reward = -113.922935, length = 65

 99%|█████████▊| 69/70 [00:00<00:00, 250.29it/s]


[Episode  201] total reward = -114.780693, length = 45
[Episode  202] total reward = -74.661919, length = 1600
[Episode  203] total reward = -116.780289, length = 106
[Episode  204] total reward = -115.490883, length = 123
[Episode  205] total reward = -107.334503, length = 44
[Episode  206] total reward = -115.129921, length = 69
[Episode  207] total reward = -108.348572, length = 88
[Episode  208] total reward = -116.296417, length = 92
[Episode  209] total reward = -113.881073, length = 108
[Episode  210] total reward = -113.793602, length = 125
[Episode  211] total reward = -122.296768, length = 180
[Episode  212] total reward = -116.237579, length = 74
[Episode  213] total reward = -114.821907, length = 91
[Episode  214] total reward = -113.709229, length = 170
[Episode  215] total reward = -118.929878, length = 118
[Episode  216] total reward = -115.741150, length = 164
[Episode  217] total reward = -114.143173, length = 79
[Episode  218] total reward = -116.044754, length = 74
[

...

[Episode 4401] total reward = 250.968903, length = 1140
[Episode 4402] total reward = 252.157700, length = 1127
[Episode 4403] total reward = -47.570183, length = 232
[Episode 4404] total reward = -48.505695, length = 236
[Episode 4405] total reward = 17.553383, length = 519
[Episode 4406] total reward = 255.081726, length = 1076
[Episode 4407] total reward = -1.770920, length = 408
[Episode 4408] total reward = -89.318306, length = 214
[Episode 4409] total reward = 41.286392, length = 573
[Episode 4410] total reward = -58.323631, length = 272
[Episode 4411] total reward = 103.912895, length = 912
[Episode 4412] total reward = -29.116005, length = 361
[Episode 4413] total reward = 259.175293, length = 1050
[Episode 4414] total reward = 104.038742, length = 806
[Episode 4415] total reward = -35.848656, length = 438
[Episode 4416] total reward = -20.129868, length = 362
[Episode 4417] total reward = -129.176743, length = 125
[Episode 4418] total reward = 260.418579, length = 1012
[Episod

100%|██████████| 402/402 [00:01<00:00, 240.96it/s]


[Episode 4601] total reward = -26.217060, length = 384
[Episode 4602] total reward = 15.746296, length = 546
[Episode 4603] total reward = 103.324387, length = 870
[Episode 4604] total reward = 261.565704, length = 1021
[Episode 4605] total reward = 92.563553, length = 772
[Episode 4606] total reward = 132.895370, length = 957
[Episode 4607] total reward = -39.401783, length = 354
[Episode 4608] total reward = 257.718353, length = 1070
[Episode 4609] total reward = -63.037132, length = 316
[Episode 4610] total reward = 10.036678, length = 509
[Episode 4611] total reward = -66.479141, length = 185
[Episode 4612] total reward = 40.986740, length = 671
[Episode 4613] total reward = 1.266712, length = 438
[Episode 4614] total reward = -31.087475, length = 319
[Episode 4615] total reward = 250.049255, length = 1152
[Episode 4616] total reward = 258.742706, length = 1051
[Episode 4617] total reward = 257.053345, length = 1056
[Episode 4618] total reward = 250.825592, length = 1136
[Episode 4

100%|██████████| 1008/1008 [00:04<00:00, 239.38it/s]


[Episode 4801] total reward = 257.988037, length = 1067
[Episode 4802] total reward = 259.735229, length = 1046
[Episode 4803] total reward = 156.606216, length = 1043
[Episode 4804] total reward = 143.503250, length = 1035
[Episode 4805] total reward = -46.008705, length = 251
[Episode 4806] total reward = 260.293640, length = 1033
[Episode 4807] total reward = 251.653137, length = 1126
[Episode 4808] total reward = -79.600502, length = 139
[Episode 4809] total reward = 22.977398, length = 509
[Episode 4810] total reward = 87.512497, length = 794
[Episode 4811] total reward = 258.719391, length = 1058
[Episode 4812] total reward = 4.547318, length = 451
[Episode 4813] total reward = -53.362183, length = 239
[Episode 4814] total reward = 256.162903, length = 1082
[Episode 4815] total reward = 252.736618, length = 1123
[Episode 4816] total reward = 20.940910, length = 561
[Episode 4817] total reward = -0.508469, length = 506
[Episode 4818] total reward = 76.925415, length = 795
[Episode

### 5.3 Load the Model and Play

In [15]:
save_dir = project_root + '/save'
model_path = os.path.join(save_dir, "model.pt")

if os.path.exists(model_path):
    print("Loading the model ... ", end="")
    checkpoint = torch.load(model_path)
    policy_net.load_state_dict(checkpoint["PolicyNet"])
    print("Done.")
else:
    print('ERROR: No model saved')

play(policy_net)

Loading the model ... Done.
[Evaluation] Total reward = 85.513852, length = 758


100%|█████████▉| 759/760 [00:02<00:00, 257.18it/s]
